In [1]:
from __future__ import annotations # To prevent Subscript for class "list" will generate runtime exception; enclose type annotation in quotes
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from Kmeans import *
%matplotlib inline
pd.options.display.max_rows = 4000

In [2]:
df = pd.read_csv('preprocessed.csv',index_col=0)
df

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,MovieID_127,MovieID_3382,MovieID_1843,MovieID_286,MovieID_3530,MovieID_2198,MovieID_2703,MovieID_2845,MovieID_3607,MovieID_2909
0,0,0.017857,0,0,0,1,0,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
1,1,1.000000,0,0,0,0,0,0,1,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
2,1,0.446429,0,0,0,0,1,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
3,1,0.803571,0,1,0,0,0,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.500000,0.5,0.0,1.0,0.75
4,1,0.446429,0,0,0,0,1,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0,0.446429,0,0,1,0,0,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
6036,0,0.803571,0,0,0,0,0,0,1,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
6037,0,1.000000,1,0,0,0,0,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75
6038,0,0.803571,1,0,0,0,0,0,0,0,...,0.0,1.0,0.5,0.5,0.75,0.845197,0.5,0.0,1.0,0.75


In [3]:
from SVD import *
u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),0,0)

KeyboardInterrupt: 